# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer 
from collections import Counter

In [2]:
data = pd.read_csv('/datasets/data.csv')
data = data.rename(columns={'debt':'credit_fail', 'income_type':'type', 'total_income':'salary'})

display(data.head(30))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,type,credit_fail,salary,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data[(data['salary'].isnull() == True) & (data['days_employed'].isnull() == True)].info()
data[(data['salary'].isnull() == True) & (data['days_employed'].isnull() == True)]['type'].value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
type                2174 non-null object
credit_fail         2174 non-null int64
salary              0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: type, dtype: int64

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
type                21525 non-null object
credit_fail         21525 non-null int64
salary              19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
print("Кол-во строк с 'days_employed' > 0 =", data[data['days_employed'] > 0].shape[0])
print("Кол-во строк с 'days_employed' < 0 =", data[data['days_employed'] < 0].shape[0])
data_pensioner = data[(data['days_employed'] > 0) & (data['type'] == 'пенсионер')]
print("Кол-во пенсионеров с 'days_employed' > 0 =", data_pensioner.shape[0])
print("Кол-во пенсионеров с 'days_employed' > 0 и кол-вом отработанных лет > 20 =", data_pensioner[data_pensioner['days_employed'] > 33000].shape[0])
print("Среднее кол-во отработанных дней у пенсионеров =", data_pensioner['days_employed'].mean())

Кол-во строк с 'days_employed' > 0 = 3445
Кол-во строк с 'days_employed' < 0 = 15906
Кол-во пенсионеров с 'days_employed' > 0 = 3443
Кол-во пенсионеров с 'days_employed' > 0 и кол-вом отработанных лет > 20 = 3443
Среднее кол-во отработанных дней у пенсионеров = 365003.4912448612


In [6]:
print("Кол-во людей с нулевым возрастом =", data[data['dob_years'] == 0].count()[0])
print("Кол-во людей с -1 ребенком =", data[data['children'] == -1].count()[0])
print("Кол-во строк с 20 детьми =", data[data['children'] == 20].count()[0])
print("Кол-во уникальных людей с 20 детьми =", len(data[data['children'] == 20]['salary'].unique()))

Кол-во людей с нулевым возрастом = 101
Кол-во людей с -1 ребенком = 47
Кол-во строк с 20 детьми = 76
Кол-во уникальных людей с 20 детьми = 68


**Вывод**

 Какие были обнаружены ошибки:
1. Минимальное значение детей "-1". Странно, что человек должен кому то ребенка) Странно что такая информация есть при оформлении кредита. Надо разобраться, возможно ошибка.
2. Максимальное значение 20 детей? Я конечно всё понимаю, но что это за мать-героиня?) Скорее всего и тут ошибка.
3. Отрицательные значения количества отработанных дней 'days_employed'. Возможно необходимо брать по модулю.
4. Среднее значенее количество отработанных дней 'days_employed' 63046 дней, а это почти 174 года, вот я им не завидую конечно) Видимо очередная пенсионная реформа в деле)
5. Возраст по некоторым строкам 'dob_years' 0. Сомневаюсь, что груднички берут кредиты. Как могут быть такие ошибки при заполнении? Просто ужас. Надо исправлять.\
6. В столбце образование имеются разные заполнения данных "Среднее", "среднее". Необходимо привести к единому стилю.
7. В столбце число проработанных днеё есть пропуски, надо исправлять. Нельзя давать кредиты безработным.
8. В столбце зарплата есть пропуски. На какие средства они кредит то платят?
9. В строках, где отсутствуют данные в столбце 'days_employed', отсутствуют данные и по 'salary'.
10. После проверки оказалось, что люди из разных профессий и взять среднее по одной группе профессий ('type') нельзя, будем заполнять NaN по среднему из каждой группы.

 Выводы:
1. Около 2/3 значений из 'days_employed' отрицательные и не относятся к пенсионерам,эту ошибку можем убрать взяв данные по модулю.
2. Практически все значения > 0 это пенсионеры, причем по всем из них стаж работы более 90 лет. Что это за роботяги такие?
3. У некоторых клиентов нулевой возраст. Данные скорее всего не были внесены в базу. Мое категоричное Фи! Этим горе кредиторам.
4. Надо исправить людей, которые задолжали ребенка. Поднимем демографию!
5. Семьи герои с 20 детьми тоже так себе информация, надо исправить.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [7]:
data_grouped = data.groupby('type').agg({'days_employed':['count', 'mean', lambda x: sum(x>0)]})

dict_to_rename = dict(zip(data_grouped.columns.levels[1], ['Общее кол-во строк', 'Среднее значение', 'Кол-во > 0']))

data_grouped = data_grouped.rename(columns=dict_to_rename, level=1)
data_grouped

days_employed                            
                Общее кол-во строк Среднее значение Кол-во > 0
type                                                          
безработный                      2    366413.652744        2.0
в декрете                        1     -3296.759962        0.0
госслужащий                   1312     -3399.896902        0.0
компаньон                     4577     -2111.524398        0.0
пенсионер                     3443    365003.491245     3443.0
предприниматель                  1      -520.848083        0.0
сотрудник                    10014     -2326.499216        0.0
студент                          1      -578.751554        0.0

**Что сделал:**
1. Сделал pivot_table по каждому из типов 'type' для того, чтоб разобраться с отрицательным стажем и понять, почему такие странные значения среднего стажа.
2. Для каждого type вывел таблицу для наглядности, в которой сделал вывод что значения 'days_employed' однотипны по пенсионерам и безработным и они > 0, а остальные меньше, соответственно пользуясь и твоим советом в том числе буду использовать abs().

In [8]:
data[['salary', 'days_employed']] = data[['salary', 'days_employed']].abs()
data['ratio_days_employed'] = data[data['days_employed'].notnull()]['days_employed']/((data['dob_years']-16)*365)

data_grouped

days_employed                            
                Общее кол-во строк Среднее значение Кол-во > 0
type                                                          
безработный                      2    366413.652744        2.0
в декрете                        1     -3296.759962        0.0
госслужащий                   1312     -3399.896902        0.0
компаньон                     4577     -2111.524398        0.0
пенсионер                     3443    365003.491245     3443.0
предприниматель                  1      -520.848083        0.0
сотрудник                    10014     -2326.499216        0.0
студент                          1      -578.751554        0.0

**Что сделал:**
1. Взял значения по 'days_employed' и 'salary' по модулю.
2. Добавил 'ratio_days_employed' чтобы вычислить полноту отработанных дней, уточнил что по закону можно работать с 16,поэтому поставил -16, чтобы потом заполнить NaN по 'days_employed'.
3. Попробовал избегать столбцы, где 'days_employed' NaN через notnull().

In [9]:
#data['days_employed'] = data.groupby('type')['days_employed'].transform(lambda x: x.fillna(x.mean()*data['dob_years']*365))
#data['salary'] = data.groupby('type')['salary'].transform(lambda x: x.fillna(x.mean()))
data['salary'] = data['salary'].fillna(data.groupby('type')['salary'].transform('median'))
data['days_employed'] = data['days_employed'].fillna(data.groupby('type')['days_employed'].transform('median'))
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,type,credit_fail,salary,purpose,ratio_days_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,0.889112
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,0.551343
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,0.906273
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,0.706292
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,25.195563


**Что сделал:**
1. Заполнил пропуски NaN в датафрейме data.
Для'days_employed' NaN заменил на произведение среднего коэффициента по группе 'type', умноженного на возраст в днях,
а в'salary' NaN заменил на среднее значение по зарплате в группе 'type'.

In [10]:
data = data.drop('ratio_days_employed', axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
type                21525 non-null object
credit_fail         21525 non-null int64
salary              21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [11]:
display(data.head(30))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,type,credit_fail,salary,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Вышеуказанными действиями выполнил предобработку данных, заменив пустые значения.

**Возможные причины пропуска данных**
1. Невнимательность оператора.
2. Ошибки, возникшие при копировании данных.
3. Также, ошибки могли возникнуть при переводе из таблиц, типа Exel.
4. Заполняли разные люди, которые как художник "Я так вижу". Каждый по своему, из за этого могло появиться, что одни и те же данные интерпритируются по разному.

### Замена типа данных

In [12]:
data['education'] = data['education'].str.lower()
print(data['education'])

0         высшее
1        среднее
2        среднее
3        среднее
4        среднее
          ...   
21520    среднее
21521    среднее
21522    среднее
21523    среднее
21524    среднее
Name: education, Length: 21525, dtype: object


**Что сделал** 

Столбец 'education' привел к единому виду

In [13]:
print("Кол-во строк с 'children' -1 =", data[data['children'] == -1].count()[0])
print("Кол-во строк с 'children' 0 =", data[data['children'] == 0].count()[0])
print("Кол-во строк с 'children' 20 =", data[data['children'] == 20].count()[0])

Кол-во строк с 'children' -1 = 47
Кол-во строк с 'children' 0 = 14149
Кол-во строк с 'children' 20 = 76


**Далее решил разобраться с детьми.** 

Видно, что всего 47 строк со значением по количеству детей < 0. Скорее всего поменяю на 0, будет по человечески, видимо изначально именно такой смысл и закладывали.
Для тех у кого 20 детей думаю поменять просто на многодетных, тоесть 3 и более. Так будет целесообразнее я считаю.

In [14]:
data['children'] = data['children'].replace(-1, 0)

In [15]:
print("Кол-во строк с 'dob_years' 0 =", data[data['dob_years'] == 0].count()[0])
print("Кол-во строк с 'dob_years' < 19 =", data[data['dob_years'] < 19].count()[0])

Кол-во строк с 'dob_years' 0 = 101
Кол-во строк с 'dob_years' < 19 = 101


**Что сделал** 

Видно, что количество одних и других совпадает соответственно это одни и те же люди

In [16]:
data['dob_years'] = data.groupby('type')['dob_years'].transform(lambda x: x.replace(0, int(x.mean())))
print("Кол-во строк с 'dob_years' 0 =", data[data['dob_years'] == 0].count()[0])
print()
data.describe()

Кол-во строк с 'dob_years' 0 = 0



,children,days_employed,dob_years,education_id,family_status_id,credit_fail,salary
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.541092,67299.486032,43.495238,0.817236,0.972544,0.080883,1.652253e+05
std,1.379943,139401.804684,12.230322,0.548138,1.420324,0.272661,9.804367e+04
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1025.549623,34.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,1993.522017,43.000000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,5347.024506,53.000000,1.000000,1.000000,0.000000,1.955499e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Что сделал** 

Заполнил количество строк с 'dob_years'. Видно что их стало 0.

In [17]:
data['days_employed'] = data['days_employed'].astype('int')
data['salary'] = data['salary'].astype('int')
data.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
type                object
credit_fail          int64
salary               int64
purpose             object
dtype: object

**Что сделал** 

1. 'salary' и 'days_employed' перевел в 'int'.
2. Использовал функцию 'astype', т.к. столбцы у нас имеют формат 'float64', т.е. перевод в 'int' сделать возможно.
3. Вижу, что 'salary' имеет вещественный тип данных, вроде бы можно менять на целочисленный через to_numeric(), но почему то были ошибки, сделал через astype.

**Вывод:**
1. Исправил'days_employed' по пенсионерам > 90 лет.Всех пенсионеров со стажем 60 лет и 6000 лет отнес в одну категорию.
2. Столбец 'education' привел к единому виду.
3. Исправил минимальное значение по 'children' с -1 на 0.
3. Исправил нулевой возраст в 'dob_years'.

### Обработка дубликатов

In [18]:
data.duplicated().sum()

71

In [19]:
data[data.duplicated(keep=False)].sort_values(by=['salary', 'days_employed'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,type,credit_fail,salary,purpose
1005,0,365213,62,среднее,1,женат / замужем,0,F,пенсионер,0,118514,ремонт жилью
1191,0,365213,61,среднее,1,женат / замужем,0,F,пенсионер,0,118514,операции с недвижимостью
1511,0,365213,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование
1681,0,365213,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
2052,0,365213,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
16148,0,1547,45,среднее,1,гражданский брак,1,F,компаньон,0,172357,свадьба
17379,0,1547,54,высшее,0,женат / замужем,0,M,компаньон,0,172357,операции с коммерческой недвижимостью
17774,1,1547,40,среднее,1,гражданский брак,1,F,компаньон,0,172357,строительство жилой недвижимости
19369,0,1547,45,среднее,1,гражданский брак,1,F,компаньон,0,172357,свадьба


In [20]:
data = data.drop_duplicates()

In [21]:
data.duplicated().sum()

0

**Вывод**

Видимо на каком то этапе произошло задвоение данных, поэтому необходимо было избавиться от дубликантов, что я и сделал.

### Лемматизация

In [22]:
unique_purpose = data['purpose'].value_counts().index.tolist()
print(data['purpose'].value_counts().to_frame())

                                        purpose
свадьба                                     791
на проведение свадьбы                       768
сыграть свадьбу                             765
операции с недвижимостью                    675
покупка коммерческой недвижимости           661
операции с жильем                           652
покупка жилья для сдачи                     651
операции с коммерческой недвижимостью       650
жилье                                       646
покупка жилья                               646
покупка жилья для семьи                     638
строительство собственной недвижимости      635
недвижимость                                633
операции со своей недвижимостью             627
строительство жилой недвижимости            624
покупка недвижимости                        621
покупка своего жилья                        620
строительство недвижимости                  619
ремонт жилью                                607
покупка жилой недвижимости              

**Что сделал:**

В датафрейме есть столбец 'purpose', поясняющий цель получения кредита, в котором содержатся похожие друг на друга цели, но выраженные разными словами. Далее выясню количество уникальных лемм и считаю необходимым добавить столбец с категорией цели кредита.

In [23]:
function = Mystem()
list_of_lemmas = []
for element in data['purpose']:
    lemma = function.lemmatize(element)
    list_of_lemmas.extend(lemma)

unique_lemmas = Counter(list_of_lemmas)
sorted(unique_lemmas.items(), key = lambda pair: pair[1], reverse=True)

[(' ', 33570),
 ('\n', 21454),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

**Что сделал**

1. Воспользовался модулем pymystem3 и лемматизиров полученный выше список, получив уникальные лемы.
2. Создал функцию для получения уникальных лемм.
3. Вижу, что есть слишком обзие понятия, такие как "покупка", "строительство", "операция", судя пор списку они и так относятся к жилью, не за чем двоить понятия.
4. Обнаружил много предлогов "с", "на", "для" и т.д., считаю необходимым и достаточным оставить слова с 4 и более буквами в дальнейшем, чтобы избавиться от предлогов.
5. В следующем пункте сделаю новый список, оставлю самые основные значения, думаю 8 штук будет достаточно.


In [24]:
final_list_purposes = [k for k in sorted(unique_lemmas, key=unique_lemmas.get, reverse=True)
                          if len(k) > 4 if k != ' ' if k != '\n'][0:8]
print(final_list_purposes)

final_list_purposes.remove('покупка') 
final_list_purposes.remove('строительство')
final_list_purposes.remove('операция')
print(final_list_purposes)

russian_stemmer = SnowballStemmer('russian')

purposes = [russian_stemmer.stem(word) for word in final_list_purposes]
print(purposes)

['недвижимость', 'покупка', 'жилье', 'автомобиль', 'образование', 'операция', 'свадьба', 'строительство']
['недвижимость', 'жилье', 'автомобиль', 'образование', 'свадьба']
['недвижим', 'жил', 'автомобил', 'образован', 'свадьб']


**Что сделал**

1. Сделал новый список final_list_purposes.
2. Сделал remove() для категорий, относящихся к строительству.
3. Вычленил основы слов, чтобы грамотно образовать категории (без ошибок).
4. Воспользовался методом стремминга, потому что в уроках было сказано, что они взаимозаменяемы, в данном случае стремминг считаю удобнее.

In [25]:
dict_purposes = {v:k for k,v in enumerate(purposes)}
dict_purposes['недвижим'] = 1
print(dict_purposes)

{'недвижим': 1, 'жил': 1, 'автомобил': 2, 'образован': 3, 'свадьб': 4}


**Что сделал:**

1. Вывел список целей в виде словаря "цель: номер типа данных"

**Выводы:**

В столбце purposes было обнаружено большое количество похожих друг на друга целей, но выраженных разными словами. Таким образом для решения данной проблемы необходимо воспользоваться методом лемматизации (в моем случае стремминг, так как они взаимозаменяемы). Благодаря данному методу удалось избавиться от проблоемы схожести целей и был создан единый и понятный список целей в виде словаря.

### Категоризация данных

In [37]:
#estate = ['недвижим', 'жиль']
#wedding = ['свадьб']
#auto = ['автомоб']
#education = ['образова']

data['purpose_category'] = 0
def categoryzer(row):
    lemmas = function.lemmatize(row)
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'образование'in lemmas:
        return 'образование'
    if ('недвижимость' in lemmas) or ('жилье' in lemmas):
        return 'недвижимость'    
    else:
        return 'другое' 

data['purpose_category'] = data['purpose'].apply(categoryzer)  
print(data['purpose_category'].unique())

['недвижимость' 'автомобиль' 'образование' 'свадьба']


**Что сделал:**

1. Создал новый столбец 'purpose_category' с категориями по целям получения кредита 'purpose' на основе созданного ранее словаря.

In [27]:
print(data[data['purpose_category'] == 0].count().sum())
print()
print(data.groupby(['education', 'education_id']).size().to_frame('count').reset_index())
print()
print(data.groupby(['family_status', 'family_status_id']).size().to_frame('count').reset_index())

0

             education  education_id  count
0               высшее             0   5250
1            начальное             3    282
2  неоконченное высшее             2    744
3              среднее             1  15172
4       ученая степень             4      6

           family_status  family_status_id  count
0  Не женат / не замужем                 4   2810
1              в разводе                 3   1195
2         вдовец / вдова                 2    959
3       гражданский брак                 1   4151
4        женат / замужем                 0  12339


**Что сделал:**

1. Проверил, есть ли нули в новом столбце. Всё нормально.
2. Проверил соответствие и уникальность пар education - education_id и family_status - family_status_id. Пары уникальны, повторов нет, соответственно они в дальнейшем проблем для категоризации не создадут.
3. Дальше чтобы ответить на множетсво вопросов и для удобства восприятия таблицы необходимо провести категоризацию. Считаю необходимым вывести в категории кол-во детей, доход, стаж и возраст. Далее напишем для них функции.

In [28]:
def salary_category(row):
    if row['salary'] <= 50000:
        return 'низкий доход'
    elif 50000 < row['salary'] <= 150000:
        return 'средний доход'
    elif 150000 < row['salary'] < 500000:
        return 'высокий доход'
    else:
        return 'очень высокий доход'

    
def days_employed_category(row):
    if row['days_employed'] <= 1827:
        return 'стаж до 5 лет'
    elif 1827 < row['days_employed'] <= 3652:
        return 'стаж 5-10 лет'
    elif 3652 < row['days_employed'] <= 7305:
        return 'стаж 10-20 лет'
    elif 7305 < row['days_employed'] <= 10950:
        return 'стаж 20-30 лет'
    else:
        return 'стаж более 30 лет'


def dob_years_category(row):
    if 18<=row['dob_years'] < 25:
        return '18 -25 лет'
    elif  25<= row['dob_years'] < 30:
        return '25-30 лет'
    elif  30<= row['dob_years'] < 35:
        return '30-35 лет'
    elif 35 <= row['dob_years'] < 40:
        return '35-40 лет'
    elif 40 <= row['dob_years'] < 50:
        return '40-50 лет'
    elif 50 <= row['dob_years'] < 65:
        return '50-65 лет'
    else:
        return 'старше 65 лет'

    
def children_category(row):
    if row['children'] == 0:
        return 'нет детей'
    elif 1 <= row['children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетный(ая)'
    
data['salary_category'] = data.apply(salary_category, axis=1)
data['days_employed_category'] = data.apply(days_employed_category, axis=1)
data['dob_years_category'] = data.apply(dob_years_category, axis=1)
data['children_category'] = data.apply(children_category, axis=1)

data.head(30)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,type,credit_fail,salary,purpose,purpose_category,salary_category,days_employed_category,dob_years_category,children_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,высокий доход,стаж 20-30 лет,40-50 лет,1-2 ребенка
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний доход,стаж 10-20 лет,35-40 лет,1-2 ребенка
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,средний доход,стаж 10-20 лет,30-35 лет,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий доход,стаж 10-20 лет,30-35 лет,многодетный(ая)
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,высокий доход,стаж более 30 лет,50-65 лет,нет детей
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,высокий доход,стаж до 5 лет,25-30 лет,нет детей
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,высокий доход,стаж 5-10 лет,40-50 лет,нет детей
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,средний доход,стаж до 5 лет,50-65 лет,нет детей
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,средний доход,стаж 10-20 лет,35-40 лет,1-2 ребенка
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,средний доход,стаж 5-10 лет,40-50 лет,нет детей


**Что сделал:**

    1. Провел категоризацию по заработной плате (к сожалению у нас практически отсутствует понятие средний класс) поэтому выбрал разбиение по з/п по наиболее встречающимуся делению.
    2. Разбил стаж по годам, сначала по 5 лет, потом по 10, так как считаю 10 лет уже достаточный стаж, можно по сильно не мельчить (если честно не знаю как разбивают работодатели, смотрю везде по разному почему то)
    3. Разбил возраст по годам, соответственно кредит в РФ можно брать с 18 лет, поэтому начал именно с 18. Далее деление по 5 и по 10 лет соответственно.
    4. Ну и наконец дети, 0 детей, до 2 и 3 и более.
    
**Выводы**

Провел категоризации по наиболее интересным с моей точки зрения позициям, сделал чтобы они были максимально информативны и сотруднику банка можно было оперативно определить все риски не отвлекаясь на цифры.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [29]:
def relation(category):
    return data.groupby(category)['credit_fail'].mean().to_frame().sort_values(by='credit_fail')
relation('children_category')

,credit_fail
children_category,
нет детей,0.075258
многодетный(ая),0.085526
1-2 ребенка,0.093003


**Вывод**

Как видим из таблицы, если нет детей, то вероятность просрочки меньшне, чем в остальных категориях. 

Ответ: да

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [30]:
#relation('family_status')
debt_from_children = pd.DataFrame()
debt_from_children['count_children'] = data.groupby('children_category')['credit_fail'].count()
debt_from_children['sum_children'] = data.groupby('children_category')['credit_fail'].sum()
debt_from_children['result_children'] = debt_from_children['sum_children'] / debt_from_children['count_children'] 
debt_from_children.sort_values('result_children', ascending = False)

,count_children,sum_children,result_children
children_category,,,
1-2 ребенка,6860,638,0.093003
многодетный(ая),456,39,0.085526
нет детей,14138,1064,0.075258


**Вывод**

Те люди, которые не имеют либо не имели законных брачных отношений, более склонны к просрочкам.

Ответ: да

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [31]:
#relation('salary_category')
debt_from_salary_category = pd.DataFrame()
debt_from_salary_category['count_salary_category'] = data.groupby('salary_category')['credit_fail'].count()
debt_from_salary_category['sum_salary_category'] = data.groupby('salary_category')['credit_fail'].sum()
debt_from_salary_category['result_salary_category'] = debt_from_salary_category['sum_salary_category'] / debt_from_salary_category['count_salary_category'] 
debt_from_salary_category.sort_values('result_salary_category', ascending = False)

,count_salary_category,sum_salary_category,result_salary_category
salary_category,,,
средний доход,11251,955,0.084881
высокий доход,9609,749,0.077948
очень высокий доход,222,14,0.063063
низкий доход,372,23,0.061828


**Вывод**

Наиболее безответственными являются заемщики со средним доходом, а с низким самые ответственные.

Ответ: да

- Как разные цели кредита влияют на его возврат в срок?

In [32]:
#relation('purpose_category')
debt_from_purpose_category = pd.DataFrame()
debt_from_purpose_category['sum_purpose_category'] = data.groupby('purpose_category')['credit_fail'].sum()
debt_from_purpose_category['count_purpose_category'] = data.groupby('purpose_category')['credit_fail'].count()
debt_from_purpose_category['result_purpose_category'] = debt_from_purpose_category['sum_purpose_category'] / debt_from_purpose_category['count_purpose_category'] 
debt_from_purpose_category.sort_values('result_purpose_category', ascending = False)

,sum_purpose_category,count_purpose_category,result_purpose_category
purpose_category,,,
автомобиль,403,4306,0.093590
образование,370,4013,0.092200
свадьба,186,2324,0.080034
недвижимость,782,10811,0.072334


**Вывод**

Таким образом, те граждане, которые вкладываются в жилье, являются самыми ответственным. Сдают видимо))

Ответ: да

## Шаг 4. Общий вывод

Таким образом получилось разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

1. Семеное положение и количество детей влияет на факт погашения кредита в строк.
2. Заемщики с официально оформленными отношениями и без детей - самые ответственные.
3. Заемщики, состоящие в неофициальном браке или находящиеся без отношений, при этом имеющие 1 или 2 детей - самые менее ответственные заемщики.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.